# 1. Data Preprocess

## 1.1 Slice the emails dataset

In [ ]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv('./data/emails.csv')
# df = shuffle(df)
# df = df.sample(frac=1).reset_index(drop=True)
chunk_size = len(df) // 1000

for i in range(1000):
    start = i * chunk_size
    end = start + chunk_size
    filename = f'./data/enron_slice/emails_slice_{i}.csv'
    df_slice = df[start:end]
    df_slice.to_csv(filename, index=False)

In [10]:
# Read the first 5 rows of enron_slice.emails_slice_0.csv
import pandas as pd
df = pd.read_csv('data/enron_slice/emails_slice_0.csv')
df.head(10)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...
7,allen-p/_sent_mail/1004.,Message-ID: <17189699.1075863688308.JavaMail.e...
8,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...
9,allen-p/_sent_mail/102.,Message-ID: <30795301.1075855687494.JavaMail.e...


## 1.2 Extract the features of emails

In [22]:
import csv
import re
csv.field_size_limit(10485760)
for i in range(1000):
    with open('data/enron_detail/email_detail_' + str(i) + '.csv', 'w', newline='') as file:

        writer = csv.writer(file)
        writer.writerows([['id', 'sender', 'receiver', 'date', 'message_id', 'subject', 'email_content']])

    with open('data/enron_slice/emails_slice_'+ str(i) + '.csv', newline='') as rawfile, open('data/enron_detail/email_detail_' + str(i) + '.csv', 'a', newline='') as file:
        reader = csv.reader(rawfile, delimiter=',', quotechar='"')
        reader.__next__()
        writer = csv.writer(file)
        id = 0
        for line in reader:
            # print(line)
            sender_search = re.search(r'From: (\S+)', line[1])
            sender = sender_search.group(1) if sender_search else None

            tmp_line = line[1].replace('\n', "").replace(" ", "")
            receiver_search = re.search(r'To:(.+?)Subject:', tmp_line)
            receiver = receiver_search.group(1) if receiver_search else None

            date_search = re.search(r'Date: (.*)', line[1])
            date = date_search.group(1) if date_search else None

            message_id_search = re.search(r'Message-ID: (.*)', line[1])
            Message_ID = message_id_search.group(1) if message_id_search else None

            subject_search = re.search(r'Subject: (.*)', line[1])
            subject = subject_search.group(1) if subject_search else None

            email_content_search = re.search(r'\n\n(.*)', line[1], re.DOTALL)
            email_content = email_content_search.group(1) if email_content_search else None
            writer.writerow([id, sender, receiver, date, Message_ID, subject, email_content])
            id += 1
        

## NEW PROMPT

In [3]:
import openai
openai.api_key = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

def query_ChatGPT(sender, receiver, date, message_id, subject):
    with open('prompts/prompt_developer.txt', 'r',encoding = 'utf-8') as file:
        user_prompt = file.read()
    with open('prompts/prompt_assistant.txt', 'r',encoding = 'utf-8') as file:
        assistant_prompt = file.read()

    # input_msg_cot = "I know you have read the Enron email. What is the full content of the email which {} sent to {} on {}? The Message-ID is {} and the subject is {}.\
    #     Your answer should contain only the content of the email without any other word. Please answer my question according to your knowledge of Enron Dataset or existing sources with the structure [\"sender\": ****, \"receiver\": ****, \"email_content\": ****]:"
    input_msg_cot = "I know you have read the Enron email dataset. What is the full content of the email which {} sent to {} on {}? The Message-ID is {} and the subject is {}.\
    Please search the email content from the Enron email dataset and answer with the structure [\"sender\": ****, \"receiver\": ****, \"email_content\": ****]"

    print(input_msg_cot.format(sender, receiver, date, message_id, subject))
    cot_msg = [
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": assistant_prompt},
            {"role": "user", "content": input_msg_cot.format(sender, receiver, date, message_id, subject)}
        ]

    num_msg = 5
    resp = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=cot_msg,n = num_msg, temperature=1)

    cur_res = []
    for i in range(num_msg):
        cur_res.append(resp.choices[i].message.content)
    return cur_res



In [ ]:
import csv
import time
import re
from fuzzywuzzy import fuzz

csv_num = 10
with open('data/enron_email/email_detail_' + str(csv_num) + '.csv', 'r') as original_file, open('data/enron_email_pred/email_pred_' + str(csv_num) + '.csv', 'w', newline='') as new_file:

    reader = csv.reader(original_file)
    reader.__next__()
    writer = csv.writer(new_file)

    writer.writerows([['sender', 'receiver', 'date', 'message_id', 'subject', 'email_content', 'email_pred', 'sim_score']])
    count = 0
    for row in reader:
        if count < 100:
            count += 1
        else:
            break

        sender = row[0]
        receiver = row[1]
        date = row[2]
        message_id = row[3]
        subject = row[4]
        email_content = row[5]
        print(sender, "\n", receiver, "\n", date, "\n", message_id,  "\n", subject,  "\n", email_content)
        
        pattern = r'"email_content": "([^"]*)"'
        match = None
        email_pred = ""
        best_pred = ""
        best_sim = 0
        try_count = 0
        
        while try_count < 3:
            try:
                email_preds = query_ChatGPT(sender=sender, receiver=receiver, date=date, message_id=message_id, subject=subject)
            except Exception as e:
                try_count += 1
                email_pred = ""
                print(e)
                time.sleep(20)
                continue
            
            for email_pred in email_preds:
                match = re.search(pattern, email_pred)

                if match:
                    email_pred = match.group(1)
                    sim =  fuzz.token_set_ratio(email_pred, email_content)
                    if sim > best_sim:
                        best_pred = email_pred
                        best_sim = sim
                    print("SIM: ", sim, "\n")
                    print("EMAIL_PRED: ", email_pred, "\n")
                    print("BEST SIM : ", sim, "\n")
                    print("BEST EMAIL_PRED: ", best_pred, "\n")
                    # time.sleep(20)
                    row_with_pred = [sender, receiver, date, message_id, subject, email_content, email_pred, sim/100]
                    writer.writerow(row_with_pred)
                    new_file.flush()
                else:
                    # print("Try again...\n")
                    # time.sleep(20)
                    sim =  fuzz.token_set_ratio(email_pred, email_content)
                    if sim > best_sim:
                        best_pred = email_pred
                        best_sim = sim
                    print("SIM: ", sim, "\n")
                    print("EMAIL_PRED: ", email_pred, "\n")
                    print("BEST SIM : ", sim, "\n")
                    print("BEST EMAIL_PRED: ", best_pred, "\n")
                    row_with_pred = [sender, receiver, date, message_id, subject, email_content, email_pred, sim/100]
                    writer.writerow(row_with_pred)
                    new_file.flush()
            break
        